# Introduction to Timeseries datasets

> Note: you will be best served by familiarizing yourself with the more basic notebooks _Introduction to Argovis_ and _Intro to Argovis' Grid API_ before following this notebook.

The generic point schema used by Argovis for point-like data such as Argo profiles and grids works well for data that can be feasilby captured as documents with unique latitude, longitude, and timestamps. However, when considering higher-resolution datasets, indexing independent documents for each such coordinate triple can dramatically exceed the scale of computing resources the point data above requires; for example, while Argo has roughly 3 million such profile documents to consider at the time of writing, a global, quarter-degree grid measured daily for 30 years (a typical scale for satellite products) would have on the order of *10 billion* such documents. In order to represent, index and serve such high-resolution grids on similar compute infrastructure to the point data, we make a minor modification to the generic point schema to form the *generic timeseries schema*:

 - Vectors in the ``data`` object represent surface measurements, estimates or flags as an ordered timeseries.
 - The ``data`` document no longer has a single ``timestamp`` key, as the data within corresponds to many timestamps.
 - The ``metadata`` or ``data`` document must bear a ``timeseries`` key, which is an ordered list of timestamps corresponding to the times associated with each element in the ``data`` vectors.

The observant reader will notice that this is very similar to the gridded products which have a ``levels`` key indicating the model depths for each entry in their ``data`` vectors. All other aspects of the generic schema remain consistent between point and timeseries datasets. In this notebook, we'll illustrate the unique features of a couple of timeseries datasets; for all other details, the reader is encouraged to apply what they learned from other Argovis API examples, as most query filters and behaviors remain identical between point and timeseries datasets.

## Setup

In addition to importing a few python packages, make sure to plug in your Argovis API key for API_KEY in the next cell. If you don't have a free Argovis API key yet, get one at https://argovis-keygen.colorado.edu/.

In [1]:
from argovisHelpers import helpers as avh
import helpers

API_ROOT='https://argovis-api.colorado.edu/'
API_KEY=''

/Users/dgiglio/opt/miniconda3/envs/argovis_demos/lib/python3.9/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


# NOAA Sea surface temperature timeseries

Argovis indexes the weekly average sea surface temperature on a 1 degree grid as provided by NOAA via [https://psl.noaa.gov/data/gridded/data.noaa.oisst.v2.html](https://psl.noaa.gov/data/gridded/data.noaa.oisst.v2.html). Let's start by having a look at the metadata for this collection:

In [ ]:
sstMetaQuery = {
    'id': 'noaasst' 
}

sstMeta = avh.query('timeseries/meta', options=sstMetaQuery, apikey=API_KEY, apiroot=API_ROOT)
print(sstMeta)

We can see from the usual `data_info` that this dataset contains one variable called `sst` corresponding to weekly mean sea surface temperature. A feature unique to timeseries datasets is that the metadata document (of which there is one per dataset) contains a `timeseries` key; this lists all the timesteps for all the timeseries in the dataset.

We can also have a look at a corresponding data document, ID'ed as `<longitude>_<latitude>`:

In [ ]:
sstQuery = {
    'id': '14.5_39.5',
    'data': 'sst'
}

sst = avh.query('timeseries/noaasst', options=sstQuery, apikey=API_KEY, apiroot=API_ROOT)
print(sst)

The `data` key here is structured according to `data_info` like all other Argovis datasets; the elements correspond to the timestamps in order as found on the metadata document.

If instead we provide a time range to our data query:

In [ ]:
sstQuery = {
    'id': '14.5_39.5',
    'data': 'sst',
    'startDate': '1993-01-01T00:00:00Z',
    'endDate': '1993-02-01T00:00:00Z'
}

sst = avh.query('timeseries/noaasst', options=sstQuery, apikey=API_KEY, apiroot=API_ROOT)
print(sst)

we get a `timeseries` key appended to the data document to indicate the timestamps of the filtered timeseries; note this is in close analogy to how levels are filtered in Argovis' representation of Argo grids, for example.

## Zonal and meridional area-weighted averages for timeseries data

Much like gridded data, timeseries data can be arranged to easily compute zonal and meridional averages, with area weighting. Lets start by downloading a year of data for a region in the North Atlantic:

In [ ]:
sstQuery = {
  "startDate": '2002-01-01T00:00:00Z',
  "endDate": '2012-01-01T00:00:00Z',
  "polygon": [[-50,50],[-50,55],[-45,55],[-45,50],[-50,50]],
  "data": 'sst'
}
sst = avh.query('timeseries/noaasst', options=sstQuery, apikey=API_KEY, apiroot=API_ROOT)

> **Temporospatial limits of timeseries queries**: because timeseries documents contain information for every timestep in the series, we currently support queries on only small geographic areas, about 50 square degrees at the equator. However, long time duration queries like the one above are well supported. You can, of course, tile multipl requests to cover an arbitrary region.

If we then arrange these data documents into a dataframe with columns for longitude, latitude, timestamp and measurement, we can compute and plot area-weighted meridional and zonal averages with our helpers:

In [ ]:
df = helpers.level_df(sst, 
                      ['sst', 'longitude', 'latitude'], 
                      timesteps=sst[0]['timeseries'], 
                      index=["latitude","longitude","timestamp"]
                     )
ds = df.to_xarray()

In [ ]:
sst_mer = helpers.regional_mean(ds, form='meridional')
sst_mer['sst'].plot(y="timestamp")

In [ ]:
sst_zon = helpers.regional_mean(ds, form='zonal')
sst_zon['sst'].plot(y="timestamp")

# Copernicus sea level anomaly timeseries

Argovis indexes a quarter-degree grid of sea level anomalies and absolute dynamic topologies from [https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-sea-level-global](https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-sea-level-global); note that the original daily data reported at this link has been averaged down to weekly averages with timestamps aligned with the NOAA SST dataset described above, for scale and comparison purposes.

Let's again start by looking at the single metadata document for this collection:

In [2]:
slaMetaQuery = {
    'id': 'copernicussla' 
}

slaMeta = avh.query('timeseries/meta', options=slaMetaQuery, apikey=API_KEY, apiroot=API_ROOT)
print(slaMeta)

[{'_id': 'copernicussla', 'data_type': 'sea level anomaly', 'data_info': [['sla', 'adt', 'ugosa', 'ugos', 'vgosa', 'vgos'], ['units', 'long_name'], [['m', 'Sea level anomaly'], ['m', 'Absolute dynamic topography'], ['m/s', 'Geostrophic velocity anomalies: zonal component'], ['m/s', 'Absolute geostrophic velocity: zonal component'], ['m/s', 'Geostrophic velocity anomalies: meridian component'], ['m/s', 'Absolute geostrophic velocity: meridian component']]], 'date_updated_argovis': '2023-11-30T21:33:53.000Z', 'timeseries': ['1993-01-03T00:00:00.000Z', '1993-01-10T00:00:00.000Z', '1993-01-17T00:00:00.000Z', '1993-01-24T00:00:00.000Z', '1993-01-31T00:00:00.000Z', '1993-02-07T00:00:00.000Z', '1993-02-14T00:00:00.000Z', '1993-02-21T00:00:00.000Z', '1993-02-28T00:00:00.000Z', '1993-03-07T00:00:00.000Z', '1993-03-14T00:00:00.000Z', '1993-03-21T00:00:00.000Z', '1993-03-28T00:00:00.000Z', '1993-04-04T00:00:00.000Z', '1993-04-11T00:00:00.000Z', '1993-04-18T00:00:00.000Z', '1993-04-25T00:00:00.000

Identical in structure to the SST metadata, though this dataset contains two data variables: the sea height anomaly `sla` as compared to the local average sea height over the reference period 1993-2012, and the absolute sea height including this anomaly, `adt`. We can query this dataset much the same as any other timeseries data:

In [3]:
slaQuery = {
    'id': '-46.875_35.625',
    'data': 'all',
    'startDate': '1993-01-01T00:00:00Z',
    'endDate': '1993-02-01T00:00:00Z'
}

sla = avh.query('timeseries/copernicussla', options=slaQuery, apikey=API_KEY, apiroot=API_ROOT)
print(sla)

[{'_id': '-46.875_35.625', 'metadata': ['copernicussla'], 'basin': 1, 'geolocation': {'type': 'Point', 'coordinates': [-46.875, 35.625]}, 'data': [[-0.20495714, -0.2092, -0.2131, -0.1916, -0.1213], [0.20065714, 0.19641429, 0.19252857, 0.21408571, 0.28434286], [0.38157143, 0.25678571, 0.23225714, 0.26804286, 0.2562], [0.31707143, 0.19225714, 0.16774286, 0.20354286, 0.19167143], [-0.23054286, -0.18638571, -0.24157143, -0.0144, 0.25228571], [-0.26914286, -0.22497143, -0.28015714, -0.053, 0.2137]], 'timeseries': ['1993-01-03T00:00:00.000Z', '1993-01-10T00:00:00.000Z', '1993-01-17T00:00:00.000Z', '1993-01-24T00:00:00.000Z', '1993-01-31T00:00:00.000Z'], 'data_info': [['sla', 'adt', 'ugosa', 'ugos', 'vgosa', 'vgos'], ['units', 'long_name'], [['m', 'Sea level anomaly'], ['m', 'Absolute dynamic topography'], ['m/s', 'Geostrophic velocity anomalies: zonal component'], ['m/s', 'Absolute geostrophic velocity: zonal component'], ['m/s', 'Geostrophic velocity anomalies: meridian component'], ['m/s',

In [4]:
slaQuery_reg = {
  "startDate": '2002-01-01T00:00:00Z',
  "endDate": '2012-01-01T00:00:00Z',
  "polygon": [[-50,50],[-50,55],[-45,55],[-45,50],[-50,50]],
  "data": 'sla'
}
sla_reg = avh.query('timeseries/copernicussla', options=slaQuery_reg, apikey=API_KEY, apiroot=API_ROOT)

In [5]:
df_sla = helpers.level_df(sla_reg, 
                      ['sla', 'longitude', 'latitude'], 
                      timesteps=sla[0]['timeseries'], 
                      index=["latitude","longitude","timestamp"]
                     )
ds_sla = df_sla.to_xarray()

ValueError: cannot convert a DataFrame with a non-unique MultiIndex into xarray

In [ ]:
ds_sla

In [ ]:
ds_sla['sla'][:,:,0].plot()

# REMSS CCMP wind vector product

Similarly to the sea level anaomaly time series, Argovis indexes a weekly average of the [REMSS CCMP wind vector product](https://www.remss.com/measurements/ccmp/). Have a look at the metadata:

In [ ]:
ccmpMetaQuery = {
    'id': 'ccmpwind' 
}

ccmpMeta = avh.query('timeseries/meta', options=ccmpMetaQuery, apikey=API_KEY, apiroot=API_ROOT)
print(ccmpMeta)

Let's find the wind data for the same location and time period as the sea surface heights above:

In [ ]:
ccmpQuery = {
    'id': '-46.875_35.625',
    'data': 'all',
    'startDate': '1993-01-01T00:00:00Z',
    'endDate': '1993-02-01T00:00:00Z'
}

ccmp = avh.query('timeseries/ccmpwind', options=ccmpQuery, apikey=API_KEY, apiroot=API_ROOT)
print(ccmp)

We can plot wind speeds in a region and on a date:

In [ ]:
params = {
  'startDate': '1993-01-01T00:00:00Z',
  'endDate': '1993-02-01T00:00:00Z',
  "polygon": [[-50,50],[-50,55],[-45,55],[-45,50],[-50,50]],
  "data": 'all'
}

wsdata = avh.query('timeseries/ccmpwind', options=params, apikey=API_KEY, apiroot=API_ROOT)

In [ ]:
df = helpers.level_df(wsdata, ['ws', 'longitude', 'latitude'], timesteps=wsdata[0]['timeseries'], index=['longitude', 'latitude', 'timestamp'])
ds = df.to_xarray()

In [ ]:
gridmap = ds.loc[{"timestamp":avh.parsetime('1993-01-10T00:00:00.000Z')}]
gridmap['ws'].plot()